In [1]:
!nvidia-smi

Sat Sep 30 11:59:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.199.02   Driver Version: 470.199.02   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 30%   58C    P0   149W / 350W |    143MiB / 24259MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from pynvml import nvmlInit, nvmlDeviceGetHandleByIndex, nvmlDeviceGetMemoryInfo

def get_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    return info.used//1024**2

In [3]:
from transformers import (
    AutoModelForCausalLM,
    BitsAndBytesConfig,
)

/home/server-ailab-12gb/miniconda3/envs/thainq_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
get_gpu_utilization()

143

In [6]:
model_name = "tiiuae/falcon-7b"

model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

The Falcon model was initialized without `trust_remote_code=True`, and will therefore leverage the transformers library implementation. tiiuae/falcon-7b's revision is set to a version that doesn't leverage remote code (4e2d06f0a7c6370ebabbc30c6f59377ae8f73d76).

In order to override this, please set a revision manually or set `trust_remote_code=True`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.89s/it]


In [7]:
get_gpu_utilization()

21040

In [4]:
model_name = "tiiuae/falcon-7b"

model = AutoModelForCausalLM.from_pretrained(model_name, load_in_8bit=True, device_map="auto")

The Falcon model was initialized without `trust_remote_code=True`, and will therefore leverage the transformers library implementation. tiiuae/falcon-7b's revision is set to a version that doesn't leverage remote code (4e2d06f0a7c6370ebabbc30c6f59377ae8f73d76).

In order to override this, please set a revision manually or set `trust_remote_code=True`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.92s/it]


In [5]:
get_gpu_utilization()

8322

In [5]:
import torch

model_name = "tiiuae/falcon-7b"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        llm_int8_threshold=6.0,
        llm_int8_has_fp16_weight=False,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
    ),
    torch_dtype=torch.bfloat16,
)

The Falcon model was initialized without `trust_remote_code=True`, and will therefore leverage the transformers library implementation. tiiuae/falcon-7b's revision is set to a version that doesn't leverage remote code (4e2d06f0a7c6370ebabbc30c6f59377ae8f73d76).

In order to override this, please set a revision manually or set `trust_remote_code=True`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.91s/it]


In [6]:
get_gpu_utilization()

4914